# Reconocimiento del nivel de Gas en medidor de tanque estacionario

## Importar Librerías

In [24]:
import sys
import cv2 as cv
import numpy as np
#from PIL import Image

## Carga de Imagen


Cargar la imagen y validar su carga <br>
<font color=yellow>**Nota:**</font> "src" será la refenrencia con la que se llamará a la imagen durante todo el proceso<br>
Para más imformación del BGR
<a>https://datahacker.rs/how-to-access-and-edit-pixel-values-in-opencv-with-python/</a>


In [25]:
#archivo a utilizar 
filename = "med_1.png"

#cv.IMREAD_COLOR comvierte la imagen se convierte en BGR
src = cv.imread(cv.samples.findFile(filename), cv.IMREAD_COLOR)

if src is None:
    print ('Error opening image!')


***

## Detección del circulo y el centro en la imagen

### Convertir la imagen en gris y quitar posible ruido en ella<br>
Se agrega un canal alpha para hacer las conversiones a grises

In [26]:
# Convertir en gris la imagen
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)

#Aplicar un filtro Gaussiano para reducir el ruido en la detección
gray = cv.medianBlur(gray, 5)

Para ver la imagen en gris

In [27]:
cv.imshow("detected circles and dial", gray)
cv.waitKey(0)

-1

## Deliminar los parametros con los que se buscara el circulo en la imagen

**gay.shape:** imagen en escala de grises<br>
**circles:** vector que almacena tres conjuntos de valores para cada circulo detectado, x_centro, y_centro, radio<br>
**HOUGH_GRADIENT:x**Define el método de detección<br>
**min_dist = gray.rows/16 :** Distancia mínima entre centros detectados.<br>
**param_1 = 200 :** Umbral superior para el detector de borde Canny interno.<br>
**param_2 = 100:** Umbral para la detección del centro.<br>
**min_radius = 0 :** Radio mínimo a detectar. <br>
**max_radius = 0 :** Radio máximo a detectar. <br>

In [28]:
rows = gray.shape[0]
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows / 8,
param1=120, param2=75,
minRadius=100, maxRadius=00)
print(circles)

[[[219.5 363.5 160.8]]]


In [29]:
edges = cv.Canny(gray,100,200)
cv.imshow("detected circles and dial", edges)
cv.waitKey(0)

-1

## Dibujar los circulos detectados

In [30]:
if circles is not None:
    circles = np.uint16(np.around(circles))
for i in circles[0, :]:
    center = (i[0], i[1])
    #center = (200,247)
 # circle center
    cv.circle(src, center, 1, (0, 0, 255), 5)
    print(center)
 # circle outline
    #radius = 90 
    radius = i[2]
cv.circle(src, center, radius, (255, 0, 255), 3)
#cv.circle(src, center, int(radius/2), (200, 0, 255), 3)
print(center[0])
print(radius)

(220, 364)
220
161


In [31]:
cv.imshow("detected circles and dial", src)
cv.waitKey(0)

-1

## Detección de la aguja del medidor

In [32]:
gray2 = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
maxValue = 255
threshold_img = 175
# redelimitar la imagen para poder identificar mejos las lineas
th, dst2 = cv.threshold(gray2, threshold_img, maxValue,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV);

#detectar los puntos que forman la linea
lines = cv.HoughLinesP(image=dst2, rho= 3, theta=np.pi/180, threshold=100, minLineLength = 15, maxLineGap = 3)

x = center[0]
y = center[1]
#print(gray2)

Visualizar la imagen con el nuevo filtro gris

In [33]:
cv.imshow("detected circles and dial", gray2)
cv.waitKey(0)

-1

Visualizar la nueva delimitación

In [34]:
cv.imshow("detected circles and dial", dst2)
cv.waitKey(0)

-1

### Calcular la distacia entre dos puntos

In [35]:
def dist_2_pts(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [36]:
diff1LowerBound = 0.25 #diff1LowerBound and diff1UpperBound determine how close the line should be from the center
diff1UpperBound = 1
diff2LowerBound = 0.5 #diff2LowerBound and diff2UpperBound determine how close the other point of the line should be to the outside of the gauge
diff2UpperBound = 1
r = radius

### Generar la linea de la aguja

In [37]:
# remover las linead de un determinado radio
final_line_list = []

in_loop = 0
        
for i in range(0, len(lines)):
    for x1, y1, x2, y2 in lines[i]:
        #print(x1, y1, x2, y2)
        diff1 = dist_2_pts(x,y, x1, y1)  # x, y is center of circle
        diff2 = dist_2_pts(x,y, x2, y2)  # x, y is center of circle

        #hacer que la primer diferencia sea la más pequeña 
        if (diff1 > diff2):
            temp = diff1
            diff1 = diff2
            diff2 = temp
                    
            # Check if line is in range of circle
        if (((diff1<diff1UpperBound*r) and (diff1>diff1LowerBound*r) and (diff2<diff2UpperBound*r)) and (diff2>diff2LowerBound*r)):
            line_length = dist_2_pts(x1, y1, x2, y2)
            final_line_list.append([x1, y1, x2, y2])
            in_loop = 1  # validar que encontro el rango del circulo para la linea

        
        if (in_loop == 1):
            # for i in range(0,len(final_line_list)):
            #     x1 = final_line_list[i][0]
            #     y1 = final_line_list[i][1]
            #     x2 = final_line_list[i][2]
            #     y2 = final_line_list[i][3]
            #     cv.line(src, (x1, y1), (x2, y2), (0, 255, 0), 2)
            n = 0
            # Asigna las coordenadas asumiendo que el primer caso es el mejor
            x1 = final_line_list[n][0]
            y1 = final_line_list[n][1]
            x2 = final_line_list[n][2]
            y2 = final_line_list[n][3]
            cv.line(src, (x1, y1), (x2, y2), (0, 255, 255), 2)
            
print(diff1, diff2) 
print(x1,y1,x2,y2)
#188 168 204 298
#178 162 182 194
#213 85 214 114
# 240 282 264 301
# 173 244 188 223



311.3213773578679 332.54323027239633
215 416 230 239


In [38]:
cv.imshow("detected circles and dial", src)
cv.waitKey(0)

### Encontrar el ángulo de inclinación

In [ ]:
# Buscar el punto más lejano del centro para determinar el ángulo
dist_pt_0 = dist_2_pts(x, y, x1, y1)
dist_pt_1 = dist_2_pts(x, y, x2, y2)

if (dist_pt_0 > dist_pt_1):
    x_angle = x1 - x
    y_angle = y - y1
else:
    x_angle = x2 - x
    y_angle = y - y2

#calcular el ángulo de inclinación con el arc tangente
res = np.arctan(np.divide(float(y_angle), float(x_angle)))

In [ ]:
# cv.imshow("detected circles and dial", src)
# cv.waitKey(0)

In [ ]:
# #Detectar el inicio
# frameHSV = cv.cvtColor(src, cv.COLOR_BGR2HSV)

# redBajo1 = np.array([0, 100, 20], np.uint8)
# redAlto1 = np.array([8, 255, 255], np.uint8)
# redBajo2 = np.array([175, 100, 20], np.uint8)
# redAlto2 = np.array([179, 255, 255], np.uint8)

# maskRed1 = cv.inRange(frameHSV, redBajo1, redAlto1)
# maskRed2 = cv.inRange(frameHSV, redBajo2, redAlto2)
# maskRed = cv.add(maskRed1, maskRed2)
# maskRedvis = cv.bitwise_and(src, src, mask= maskRed)        
# cv.imshow('frame', src)
# cv.imshow('maskRed', maskRed)
# cv.imshow('maskRedvis', maskRedvis)

In [ ]:
res = np.rad2deg(res)
if x_angle > 0 and y_angle > 0:  #in quadrant I
    final_angle = 270 - res
if x_angle < 0 and y_angle > 0:  #in quadrant II
    final_angle = 90 - res
if x_angle < 0 and y_angle < 0:  #in quadrant III
    final_angle = 90 - res
if x_angle > 0 and y_angle < 0:  #in quadrant IV
    final_angle = 270 - res

### Unidades

In [ ]:
min_angle = 10
max_angle = 350
min_value = 5
max_value = 95
units ="porciento"

### Calcular el valor


In [ ]:
old_min = float(min_angle)
old_max = float(max_angle)

new_min = float(min_value)
new_max = float(max_value)

old_value = final_angle
print(old_value)

old_range = (old_max - old_min)
new_range = (new_max - new_min)
new_value = (((old_value - old_min) * new_range) / old_range) + new_min
print(old_range, new_range,new_min )

184.57392125990086
340.0 90.0 5.0


## Mostrar los circulos

***Solo demostrativo***





In [ ]:
cv.imshow("detected circles and dial", src)
cv.waitKey(0)


-1

In [ ]:
print ("Current reading: %s %s" %(("%.2f" % new_value), units))


Current reading: 51.21 porciento
